In [3]:
import os
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.model_selection import train_test_split
import pandas as pd

seed = 42
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
base_dir = "D:\Codes\GDSC Task Round\Task 2"
data_dir = os.path.join(base_dir, 'PetImages')
all_image_files = []
for root, dirs, files in os.walk(data_dir):
    for file in files:
        if file.endswith(('.jpg', '.jpeg', '.png')):
            all_image_files.append(os.path.join(root, file))
labels = [1 if 'Dog' in file else 0 for file in all_image_files]
labels = [str(label) for label in labels]
train_files, test_files, train_labels, test_labels = train_test_split(
    all_image_files, labels, test_size=0.2, random_state=seed, stratify=labels
)
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)
batch_size = 32
train_generator = datagen.flow_from_dataframe(
    pd.DataFrame({'filename': train_files, 'class': train_labels}),
    x_col='filename',
    y_col='class',
    target_size=(150, 150),
    batch_size=batch_size,
    class_mode='binary'
)
test_generator = datagen.flow_from_dataframe(
    pd.DataFrame({'filename': test_files, 'class': test_labels}),
    x_col='filename',
    y_col='class',
    target_size=(150, 150),
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False
)
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(1, activation='sigmoid')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
epochs = 10
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=test_generator,
    validation_steps=test_generator.samples // batch_size
)
test_loss, test_acc = model.evaluate(test_generator, steps=test_generator.samples // batch_size)
print(f'Test accuracy: {test_acc * 100:.2f}%')


predictions = model.predict(test_generator)

Found 19948 validated image filenames belonging to 2 classes.
Found 4988 validated image filenames belonging to 2 classes.
Epoch 1/10
623/623 [==============================] - ETA: 0s - loss: 0.6569 - accuracy: 0.5979

C:\Users\shriv\AppData\Local\Programs\Python\Python310\lib\site-packages\PIL\TiffImagePlugin.py:864: UserWarning: Truncated File Read
  warnings.warn(str(msg))


623/623 [==============================] - 1497s 2s/step - loss: 0.6569 - accuracy: 0.5979 - val_loss: 0.6401 - val_accuracy: 0.6331
Epoch 2/10
623/623 [==============================] - 1142s 2s/step - loss: 0.5903 - accuracy: 0.6823 - val_loss: 0.5773 - val_accuracy: 0.7075
Epoch 3/10
623/623 [==============================] - 1060s 2s/step - loss: 0.5552 - accuracy: 0.7130 - val_loss: 0.5361 - val_accuracy: 0.7290
Epoch 4/10
623/623 [==============================] - 851s 1s/step - loss: 0.5330 - accuracy: 0.7265 - val_loss: 0.5576 - val_accuracy: 0.7113
Epoch 5/10
623/623 [==============================] - 860s 1s/step - loss: 0.5137 - accuracy: 0.7433 - val_loss: 0.4809 - val_accuracy: 0.7714
Epoch 6/10
623/623 [==============================] - 487s 782ms/step - loss: 0.4916 - accuracy: 0.7632 - val_loss: 0.4984 - val_accuracy: 0.7593
Epoch 7/10
623/623 [==============================] - 480s 771ms/step - loss: 0.4689 - accuracy: 0.7772 - val_loss: 0.4609 - val_accuracy: 0.7937
E